In [18]:
import pybamm
import tracemalloc
import numpy as np

model = pybamm.lithium_ion.DFN()
solver = pybamm.IDAKLUSolver()
experiment = pybamm.Experiment(
    ["Charge at 1 A for 1 hour", "Discharge at 1 A for 1 hour"] * 100
)

tracemalloc.start()
sim = pybamm.Simulation(model, solver=solver, experiment=experiment)
sol = sim.solve()
t_eval = np.linspace(0, 3600 * 10, 100)
voltage = sol["Terminal voltage [V]"](t_eval)
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics("lineno")
total_size = sum(stat.size for stat in top_stats)
print("Total allocated size: %.1f KiB" % (total_size / 1024))
tracemalloc.stop()

2025-01-08 08:51:58.686 - [WARNING] callbacks.on_experiment_infeasible_event(252): 

	Experiment is infeasible: 'event: Maximum voltage [V]' was triggered during 'Charge at 1 A for 1 hour'. The returned solution only contains up to step 1 of cycle 1. 


Total allocated size: 9599.1 KiB


In [15]:
import pybamm
import tracemalloc
import numpy as np

model = pybamm.lithium_ion.DFN()
solver = pybamm.IDAKLUSolver(output_variables=["Voltage [V]"])
experiment = pybamm.Experiment(
    ["Charge at 1 A for 1 hour", "Discharge at 1 A for 1 hour"] * 10
)

tracemalloc.start()
sim = pybamm.Simulation(model, solver=solver, experiment=experiment)
sol = sim.solve()
t_eval = np.linspace(0, 3600 * 10, 100)
voltage = sol["Voltage [V]"](t_eval)
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics("lineno")
total_size = sum(stat.size for stat in top_stats)
print("Total allocated size: %.1f KiB" % (total_size / 1024))
tracemalloc.stop()

2025-01-08 08:51:27.636 - [WARNING] callbacks.on_experiment_infeasible_event(252): 

	Experiment is infeasible: 'event: Maximum voltage [V]' was triggered during 'Charge at 1 A for 1 hour'. The returned solution only contains up to step 1 of cycle 1. 


Total allocated size: 7671.9 KiB
